In [4]:
import pandas as pd
import spacy

In [5]:
dados_treino = pd.read_csv("/content/drive/My Drive/curso_word2vec/treino1.csv")
dados_treino.sample(5)

,title,text,date,category,subcategory,link
49980,Associação LGBT vai denunciar Eduardo Cunha à OEA,"A ABGLT (Associação Brasileira de Gays, Lésbic...",2016-04-20,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
32242,Cegos se adaptam ao início de nova vida em SP ...,Na semana em que se celebra mundialmente a pes...,2016-09-12,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2016/12...
52258,Ministro da Saúde recua e diz não ser preciso ...,Após afirmar que o Estado não tem como assumir...,2016-05-17,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2016/05...
1115,Mestre do blues de Nova Orleans fará shows no ...,Um dos artistas que melhor representam a eclét...,2016-04-26,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2016/04...
50818,Petrobras retoma proposta de reduzir carga hor...,A Petrobras decidiu retomar a proposta de ofer...,2016-06-13,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/06/1...


In [6]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 64.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
nlp = spacy.load("pt_core_news_sm")


In [8]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)

In [9]:
doc


Rio de Janeiro é uma cidade maravilhosa

In [10]:
type(doc[2])


spacy.tokens.token.Token

In [11]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])


In [12]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2:
        return  " ".join(tokens_validos)

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [13]:
texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [14]:
from time import time

t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000,
                                                        n_process = -1)]

tf = time() - t0

print(tf/60)


5.448689325650533


In [16]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [17]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [2]:
w2v_modelo


In [18]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
84466


In [19]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]


In [20]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

In [21]:
dir(w2v_modelo)


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 

In [22]:
w2v_modelo.corpus_count


84466

In [23]:
w2v_modelo.train(lista_lista_tokens,
                 total_examples=w2v_modelo.corpus_count,
                 epochs = 30)

(14585129, 16207260)

In [24]:
w2v_modelo.wv.most_similar("google")


[('apple', 0.5678371787071228),
 ('facebook', 0.4938706159591675),
 ('amazon', 0.489382803440094),
 ('airbnb', 0.4799703061580658),
 ('tesla', 0.4652266502380371),
 ('uber', 0.4620467722415924),
 ('yahoo', 0.43984708189964294),
 ('waze', 0.438750296831131),
 ('fbi', 0.4364021420478821),
 ('canais', 0.4315919876098633)]

In [25]:
w2v_modelo.wv.most_similar("microsoft")


[('unilever', 0.5707045197486877),
 ('amazon', 0.5694901347160339),
 ('tesla', 0.5340434312820435),
 ('sky', 0.5278512835502625),
 ('lego', 0.509564220905304),
 ('spotify', 0.5017371773719788),
 ('viajante', 0.4965793192386627),
 ('walmart', 0.49493759870529175),
 ('canais', 0.4910546541213989),
 ('sony', 0.478291779756546)]

In [26]:
w2v_modelo.wv.most_similar("barcelona")


[('bayern', 0.5563793182373047),
 ('psg', 0.5497195720672607),
 ('barça', 0.5424633622169495),
 ('united', 0.5251541137695312),
 ('chelsea', 0.5186861753463745),
 ('munique', 0.5160678029060364),
 ('atlético', 0.5132724046707153),
 ('figueirense', 0.509124219417572),
 ('leicester', 0.5077039003372192),
 ('madrid', 0.5060722827911377)]

In [27]:
w2v_modelo.wv.most_similar("messi")


[('suárez', 0.5682366490364075),
 ('enrique', 0.5228801965713501),
 ('cristiano', 0.5185950994491577),
 ('tevez', 0.5101930499076843),
 ('cavani', 0.5072651505470276),
 ('benzema', 0.505926787853241),
 ('neymar', 0.5049655437469482),
 ('barça', 0.5029101967811584),
 ('psg', 0.49954769015312195),
 ('mordida', 0.48260629177093506)]

In [33]:
w2v_modelo.wv.most_similar("jesus")


[('garcía', 0.527075469493866),
 ('marquinhos', 0.5055131912231445),
 ('neon', 0.49458858370780945),
 ('gabriel', 0.4910062849521637),
 ('lucas', 0.46779865026474),
 ('aranha', 0.45786750316619873),
 ('márquez', 0.45773959159851074),
 ('dudu', 0.44902274012565613),
 ('arouca', 0.43920204043388367),
 ('imagina', 0.4252317547798157)]

In [28]:
w2v_modelo.wv.most_similar("gm")


[('chrysler', 0.6750116348266602),
 ('volks', 0.6463137865066528),
 ('honda', 0.6354207396507263),
 ('embraer', 0.623967170715332),
 ('braskem', 0.6118649840354919),
 ('tesla', 0.5988661646842957),
 ('toyota', 0.5939783453941345),
 ('fiat', 0.5806877613067627),
 ('csn', 0.579312801361084),
 ('volkswagen', 0.5698086619377136)]

In [35]:
#Treinamento do modelo Skip-Gram
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs = 30)

(14584434, 16207260)

In [36]:
w2v_modelo_sg.wv.most_similar("google")


[('reguladores', 0.4334799647331238),
 ('android', 0.42049872875213623),
 ('yahoo', 0.40409713983535767),
 ('facebook', 0.4036314785480499),
 ('anunciantes', 0.3982723355293274),
 ('apple', 0.3918571472167969),
 ('buffett', 0.39089152216911316),
 ('waze', 0.3902689516544342),
 ('verizon', 0.3702603578567505),
 ('patentes', 0.35894662141799927)]

In [37]:
w2v_modelo.wv.most_similar("google")


[('apple', 0.5678371787071228),
 ('facebook', 0.4938706159591675),
 ('amazon', 0.489382803440094),
 ('airbnb', 0.4799703061580658),
 ('tesla', 0.4652266502380371),
 ('uber', 0.4620467722415924),
 ('yahoo', 0.43984708189964294),
 ('waze', 0.438750296831131),
 ('fbi', 0.4364021420478821),
 ('canais', 0.4315919876098633)]

In [38]:
w2v_modelo_sg.wv.most_similar("gm")


[('metalúrgicos', 0.5724930763244629),
 ('motors', 0.5253599882125854),
 ('audi', 0.4895733892917633),
 ('honda', 0.4873236417770386),
 ('bmw', 0.4814101457595825),
 ('mitsubishi', 0.4752597510814667),
 ('airbags', 0.47330546379089355),
 ('chrysler', 0.46469783782958984),
 ('autoguiados', 0.45922937989234924),
 ('volkswagen', 0.4587068259716034)]

In [39]:
w2v_modelo.wv.most_similar("gm")


[('chrysler', 0.6750116348266602),
 ('volks', 0.6463137865066528),
 ('honda', 0.6354207396507263),
 ('embraer', 0.623967170715332),
 ('braskem', 0.6118649840354919),
 ('tesla', 0.5988661646842957),
 ('toyota', 0.5939783453941345),
 ('fiat', 0.5806877613067627),
 ('csn', 0.579312801361084),
 ('volkswagen', 0.5698086619377136)]

In [40]:
w2v_modelo.wv.save_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_skipgram.txt", binary=False)

In [41]:
!python -m spacy download pt_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 49.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [42]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_skipgram.txt")
artigo_treino = pd.read_csv("/content/drive/My Drive/curso_word2vec/treino.csv")
artigo_teste = pd.read_csv("/content/drive/My Drive/curso_word2vec/teste.csv")

In [43]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizador(texto):

    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())


    return  tokens_validos

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [44]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros((1,300))

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass


    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[ 0.45550351 -0.25076801  0.77740788 -1.54668684  0.12073335  1.37497871
  -1.91394108  0.13721597  0.59932084 -0.78771667  1.51351412  1.44695464
  -0.67971745 -0.39370942 -1.36386204 -2.16041604 -0.32573342 -2.08773178
  -0.16912091 -0.51283469  0.09683248 -0.8865231   1.10607477  0.31310403
  -0.22995839 -1.52140412  0.16254951  0.36050819 -0.67958833  1.28671777
  -0.93221813 -3.25311327 -0.59397426  1.23145093  1.86639243 -0.69045519
  -3.05274057 -2.52369371  1.00769103 -0.56080976 -0.87502609  0.52252696
   1.10800679 -0.94818813 -1.7635344  -0.73580074  1.48040935 -0.29619314
   2.8090179  -0.05333917  0.07541613 -1.86086637  1.69996019 -1.80773402
  -0.15680569  1.96782176  0.32063679 -0.85207478  1.01582667  1.29353517
  -1.51305838 -2.10148203  0.97754657 -0.93815087  1.2525984   0.89758098
   0.59169602 -0.0614945   0.80742329 -1.12763956  0.46143468  2.28939751
  -0.62757288 -1.31824831  0.52544585  1.63973451 -0.44526577 -0.5159483
  -2.21446878  1.22455196 -1.6

In [46]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

    RL = LogisticRegression(max_iter = 800)
    RL.fit(x_treino, y_treino)
    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)

    return RL

RL_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.63      0.80      0.71      1698
     esporte       0.93      0.86      0.90      4663
   ilustrada       0.13      0.85      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.84      0.78      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [49]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

RL_sg = classificador(w2v_modelo_sg,
                        matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)


              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.81      0.71      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.14      0.86      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [ ]:
import pickle

with open("/content/drive/My Drive/curso_word2vec/rl_cbow.pkl", "wb") as f:
    pickle.dump(RL_cbow, f)

In [ ]:
with open("/content/drive/My Drive/curso_word2vec/rl_sg.pkl", "wb") as f:
    pickle.dump(RL_sg, f)